In [6]:
import calendar
import gc
import os
import sys

import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandasql as ps
import scipy
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from implicit.als import AlternatingLeastSquares
import zipfile

In [7]:
os.getcwd()

'/home/mle-user/mle_projects/mle-pr-final/notebooks'

In [9]:
os.listdir('/home/mle-user/mle_projects/mle-pr-final/data')

['archive.zip']

In [10]:
arch_path = '/home/mle-user/mle_projects/mle-pr-final/data'
arch_name ='archive.zip'
dest = '/home/mle-user/mle_projects/mle-pr-final/data'

In [11]:
zip_path = os.path.join(arch_path, arch_name)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
     zip_ref.extractall(dest)

In [12]:
data_path = '/home/mle-user/mle_projects/mle-pr-final/data'
os.listdir(data_path)

['category_tree.csv',
 'item_properties_part1.csv',
 'events.csv',
 'item_properties_part2.csv',
 'archive.zip']

In [13]:
category_tree = pd.read_csv(os.path.join(data_path, 'category_tree.csv'))
events = pd.read_csv(os.path.join(data_path, 'events.csv'))
item_properties_part1 = pd.read_csv(os.path.join(data_path, 'item_properties_part1.csv'))
item_properties_part2 = pd.read_csv(os.path.join(data_path, 'item_properties_part2.csv'))

In [16]:
def explore_data(df):
    """
    Исследует структуру данных: типы данных, пропущенные значения, базовые статистики.
    """
    print("===== Основная информация о данных =====")
    print(f"Размер датасета: {df.shape}")
    print("\nТипы данных:")
    print(df.dtypes)
    
    print("\nПропущенные значения:")
    missing_values = df.isnull().sum()
    print(missing_values[missing_values > 0])
    
    print("\nБазовая статистика:")
    print(df.describe())
    print(df.head(3))

In [17]:
explore_data(category_tree)

===== Основная информация о данных =====
Размер датасета: (1669, 2)

Типы данных:
categoryid      int64
parentid      float64
dtype: object

Пропущенные значения:
parentid    25
dtype: int64

Базовая статистика:
        categoryid     parentid
count  1669.000000  1644.000000
mean    849.285201   847.571168
std     490.195116   505.058485
min       0.000000     8.000000
25%     427.000000   381.000000
50%     848.000000   866.000000
75%    1273.000000  1291.000000
max    1698.000000  1698.000000
   categoryid  parentid
0        1016     213.0
1         809     169.0
2         570       9.0


In [18]:
explore_data(events)

===== Основная информация о данных =====
Размер датасета: (2756101, 5)

Типы данных:
timestamp          int64
visitorid          int64
event             object
itemid             int64
transactionid    float64
dtype: object

Пропущенные значения:
transactionid    2733644
dtype: int64

Базовая статистика:
          timestamp     visitorid        itemid  transactionid
count  2.756101e+06  2.756101e+06  2.756101e+06   22457.000000
mean   1.436424e+12  7.019229e+05  2.349225e+05    8826.497796
std    3.366312e+09  4.056875e+05  1.341954e+05    5098.996290
min    1.430622e+12  0.000000e+00  3.000000e+00       0.000000
25%    1.433478e+12  3.505660e+05  1.181200e+05    4411.000000
50%    1.436453e+12  7.020600e+05  2.360670e+05    8813.000000
75%    1.439225e+12  1.053437e+06  3.507150e+05   13224.000000
max    1.442545e+12  1.407579e+06  4.668670e+05   17671.000000
       timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214

In [19]:
explore_data(item_properties_part1)

===== Основная информация о данных =====
Размер датасета: (10999999, 4)

Типы данных:
timestamp     int64
itemid        int64
property     object
value        object
dtype: object

Пропущенные значения:
Series([], dtype: int64)

Базовая статистика:
          timestamp        itemid
count  1.100000e+07  1.100000e+07
mean   1.435158e+12  2.333851e+05
std    3.327653e+09  1.348258e+05
min    1.431227e+12  0.000000e+00
25%    1.432436e+12  1.165150e+05
50%    1.433646e+12  2.334990e+05
75%    1.437880e+12  3.501860e+05
max    1.442113e+12  4.668660e+05
       timestamp  itemid    property                            value
0  1435460400000  460429  categoryid                             1338
1  1441508400000  206783         888          1116713 960601 n277.200
2  1439089200000  395014         400  n552.000 639502 n720.000 424566


In [20]:
explore_data(item_properties_part2)

===== Основная информация о данных =====
Размер датасета: (9275903, 4)

Типы данных:
timestamp     int64
itemid        int64
property     object
value        object
dtype: object

Пропущенные значения:
Series([], dtype: int64)

Базовая статистика:
          timestamp        itemid
count  9.275903e+06  9.275903e+06
mean   1.435156e+12  2.333968e+05
std    3.327970e+09  1.348682e+05
min    1.431227e+12  0.000000e+00
25%    1.432436e+12  1.165175e+05
50%    1.433646e+12  2.334620e+05
75%    1.437880e+12  3.504470e+05
max    1.442113e+12  4.668660e+05
       timestamp  itemid property            value
0  1433041200000  183478      561           769062
1  1439694000000  132256      976  n26.400 1135780
2  1435460400000  420307      921  1149317 1257525
